In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "deepseek-r1:14b"

In [ ]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    def __init__(self, url):
        self.url = url
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            soup = BeautifulSoup(response.content, 'html.parser')
            self.title = soup.title.string if soup.title else "No title found"
            if soup.body:
                for irrelevant in soup.body(["script", "style", "img", "input"]):
                    irrelevant.decompose()
                self.text = soup.body.get_text(separator="\n", strip=True)
            else:
                self.text = "No body content found"
        except requests.RequestException as e:
            print(f"Error fetching website: {e}")
            self.title = "Error loading page"
            self.text = "Could not load page content"

## Without Ollama package

In [ ]:
def house_renting(system_prompt, user_prompt):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]
    payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }
    response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
    return response.json()['message']['content']

## Introducing Ollama package 

In [ ]:
import ollama

def house_renting_ollama(system_prompt, user_prompt):
    try:
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        response = ollama.chat(model=MODEL, messages=messages)
        return response['message']['content']
    except Exception as e:
        return f"Error communicating with Ollama: {e}"

In [ ]:
system_prompt = """
You are a helpful real estate assistant specializing in UK property rentals. Your job is to guide users in finding houses to rent, especially in Durham. Follow these rules:
1. Always ask clarifying questions if user input is vague. Determine location, budget, number of bedrooms, and tenant type (e.g. student, family, professional).
2. Use structured data provided from the website (like property listings) to identify relevant options.
3. If listings are provided, filter and rank them based on the user's preferences.
4. Recommend up to 5 top properties with rent price, bedroom count, key features, and location.
5. Always respond in markdown with clean formatting using headers, bold text, and bullet points.
6. If no listings match well, provide tips (e.g. "try adjusting your budget or search radius").
7. Stay concise, helpful, and adapt to whether the user is a student, family, couple, or solo tenant.
"""

def user_prompt_for_renting(website, user_needs):
    return f"""
I want to rent a house and here's what I'm looking for:
{user_needs}

Here are the property listings I found on the website titled: "{website.title}".

Please analyze them and recommend the best 3–5 options that match my needs. If none are suitable, tell me why and offer suggestions.

The page content is below:
{website.text[:4000]}
"""  # content is truncated for token limits

## Ollama Package

In [ ]:
if __name__ == "__main__":  
    print("Starting AI Property Rental Assistant...")
    print("=" * 50)
    
    website_url = "https://www.onthemarket.com/to-rent/property/durham/"
    print(f"🔍 Scraping properties from: {website_url}")
    
    website = Website(website_url)
    print(f"Website Title: {website.title}")
    print(f"Content Length: {len(website.text)} characters")
    print(f"Successfully scraped property listings\n")
    
    user_needs = "I'm a student looking for a 2-bedroom house in Durham under £2,000/month"
    print(f"User Requirements: {user_needs}\n")
    
    user_prompt = user_prompt_for_renting(website, user_needs)
    print("Generating AI recommendations...")
    
    # Choose which method to use (comment out the one you don't want)
    
    # Method 1: Using ollama Python library
    output = house_renting_ollama(system_prompt, user_prompt)
    
    # Method 2: Using direct API call
    # output = house_renting(system_prompt, user_prompt)
    
    display(Markdown(output))